# Stock Price Prediction System

This notebook implements an advanced stock price prediction system using machine learning models including XGBoost ensemble and LSTM neural networks.

## Import Libraries

In [1]:
!pip install yfinance xgboost tensorflow scikit-learn pandas numpy matplotlib plotly seaborn --quiet

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, ElasticNet

In [4]:
try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    LSTM_AVAILABLE = True
    print("TensorFlow available for LSTM models")
except ImportError:
    print("TensorFlow not available. LSTM model will be skipped.")
    LSTM_AVAILABLE = False

TensorFlow available for LSTM models


In [5]:
from datetime import datetime, timedelta

## Configuration

In [6]:
# Configuration parameters
STOCK_TICKER = "AAPL"  # Default stock ticker
MODEL_CHOICE = 1  # 1: XGBoost, 2: LSTM, 3: Both
FORECAST_HORIZON = 1  # Days to predict ahead

## Data Fetching Functions

In [ ]:
## Data Fetching Functions

def fetch_stock_data(ticker_symbol, period="max"):
    """Fetch stock data from Yahoo Finance"""
    ticker = yf.Ticker(ticker_symbol)
    data = ticker.history(period=period)
    
    if data.empty:
        print(f"No data found for {ticker_symbol}")
        return None
    
    info = ticker.info
    company_name = info.get('longName', ticker_symbol)
    
    print(f"\nFetched data for {company_name} ({ticker_symbol})")
    print(f"Data range: {data.index[0].date()} to {data.index[-1].date()}")
    print(f"Total trading days: {len(data)}")
    
    return data, company_name

def get_valid_ticker():
    """Get valid ticker input from user"""
    while True:
        stock = input("Enter a valid stock ticker (e.g., AAPL, TSLA, MSFT): ").upper()
        try:
            test = yf.Ticker(stock)
            if test.history(period="1d").empty:
                print("Invalid ticker. Please try again.")
            else:
                return stock
        except Exception as e:
            print(f"Error validating ticker: {e}")
            print("Invalid input. Please try again.")

def get_model_choice():
    """Get model choice from user"""
    print("\nChoose prediction model:")
    print("1. XGBoost (Less time, High accuracy)")
    print("2. LSTM (More time, Most accuracy)")  
    print("3. Both (Conclusion with both models)")
    
    while True:
        choice = input("Enter your choice (1/2/3): ")
        if choice in ['1', '2', '3']:
            return int(choice)
        print("Invalid choice. Please enter 1, 2, or 3.")

## Data Exploration

# Test data fetching
print("Testing data fetching...")
sample_data = fetch_stock_data("AAPL", "1y")
if sample_data is not None:
    data, company_name = sample_data
    print(f"\nSample data shape: {data.shape}")
    print(f"Columns: {list(data.columns)}")
    print(f"\nFirst few rows:")
    print(data.head())
    print(f"\nLast few rows:")
    print(data.tail())
    print(f"\nBasic statistics:")
    print(data.describe())

## Basic Visualization

plt.figure(figsize=(12, 6))
if sample_data is not None:
    data, _ = sample_data
    plt.plot(data.index, data['Close'], label='Close Price', linewidth=2)
    plt.title(f'{company_name} Stock Price - Last Year')
    plt.xlabel('Date')
    plt.ylabel('Price ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization")

Testing data fetching...


$AAPL: possibly delisted; no price data found  (period=1y)


No data found for AAPL
No data available for visualization


<Figure size 1200x600 with 0 Axes>

## Technical Indicators

In [14]:
## Technical Indicators

def calculate_rsi(prices, window=14):
    """Calculate Relative Strength Index"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(prices, fast=12, slow=26, signal=9):
    """Calculate MACD indicator"""
    ema_fast = prices.ewm(span=fast).mean()
    ema_slow = prices.ewm(span=slow).mean()
    macd = ema_fast - ema_slow
    macd_signal = macd.ewm(span=signal).mean()
    macd_hist = macd - macd_signal
    return macd, macd_signal, macd_hist

def calculate_bollinger_bands(prices, window=20, num_std=2):
    """Calculate Bollinger Bands"""
    rolling_mean = prices.rolling(window=window).mean()
    rolling_std = prices.rolling(window=window).std()
    bb_upper = rolling_mean + (rolling_std * num_std)
    bb_lower = rolling_mean - (rolling_std * num_std)
    return bb_upper, rolling_mean, bb_lower

def calculate_stochastic(high, low, close, k_window=14, d_window=3):
    """Calculate Stochastic Oscillator"""
    lowest_low = low.rolling(window=k_window).min()
    highest_high = high.rolling(window=k_window).max()
    stoch_k = 100 * ((close - lowest_low) / (highest_high - lowest_low))
    stoch_d = stoch_k.rolling(window=d_window).mean()
    return stoch_k, stoch_d

def calculate_atr(high, low, close, window=14):
    """Calculate Average True Range"""
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(window=window).mean()
    return atr

def calculate_williams_r(high, low, close, window=14):
    """Calculate Williams %R"""
    highest_high = high.rolling(window=window).max()
    lowest_low = low.rolling(window=window).min()
    williams_r = -100 * ((highest_high - close) / (highest_high - lowest_low))
    return williams_r

def add_technical_indicators(data):
    """Add all technical indicators to the dataset"""
    df = data.copy()
    
    # Basic price features
    df['Returns'] = df['Close'].pct_change()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['High_Low_Pct'] = (df['High'] - df['Low']) / df['Close']
    df['Open_Close_Pct'] = (df['Close'] - df['Open']) / df['Open']
    df['Price_Volume'] = df['Close'] * df['Volume']
    df['Volume_Rate'] = df['Volume'] / df['Volume'].rolling(window=20).mean()
    
    # Moving averages
    for period in [3, 5, 10, 20, 50, 100]:
        df[f'MA_{period}'] = df['Close'].rolling(window=period).mean()
        df[f'MA_{period}_ratio'] = df['Close'] / df[f'MA_{period}']
        df[f'EMA_{period}'] = df['Close'].ewm(span=period).mean()
        df[f'EMA_{period}_ratio'] = df['Close'] / df[f'EMA_{period}']
        df[f'Close_MA_{period}_diff'] = df['Close'] - df[f'MA_{period}']
    
    # Technical indicators
    df['RSI'] = calculate_rsi(df['Close'])
    df['RSI_7'] = calculate_rsi(df['Close'], 7)
    df['RSI_21'] = calculate_rsi(df['Close'], 21)
    
    df['MACD'], df['MACD_Signal'], df['MACD_Hist'] = calculate_macd(df['Close'])
    
    df['BB_Upper'], df['BB_Middle'], df['BB_Lower'] = calculate_bollinger_bands(df['Close'])
    df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / df['BB_Middle']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    df['Stoch_K'], df['Stoch_D'] = calculate_stochastic(df['High'], df['Low'], df['Close'])
    df['ATR'] = calculate_atr(df['High'], df['Low'], df['Close'])
    df['Williams_R'] = calculate_williams_r(df['High'], df['Low'], df['Close'])
    
    # Volume and volatility indicators
    for period in [5, 10, 20]:
        df[f'Vol_MA_{period}'] = df['Volume'].rolling(window=period).mean()
        df[f'Vol_Std_{period}'] = df['Volume'].rolling(window=period).std()
        df[f'Price_Volatility_{period}'] = df['Close'].rolling(window=period).std()
        df[f'High_MA_{period}'] = df['High'].rolling(window=period).mean()
        df[f'Low_MA_{period}'] = df['Low'].rolling(window=period).mean()
    
    return df

## Test Technical Indicators

if sample_data is not None:
    data, company_name = sample_data
    print("Adding technical indicators...")
    data_with_indicators = add_technical_indicators(data)
    print(f"Dataset shape after adding indicators: {data_with_indicators.shape}")
    print(f"New columns added: {len(data_with_indicators.columns) - len(data.columns)}")
    
    # Display some indicators
    print("\nSample technical indicators:")
    indicator_cols = ['RSI', 'MACD', 'BB_Position', 'Stoch_K', 'ATR']
    available_cols = [col for col in indicator_cols if col in data_with_indicators.columns]
    if available_cols:
        print(data_with_indicators[available_cols].tail())

## Feature Engineering

In [ ]:
## Feature Engineering

def create_sequence_features(data, target_col='Close', sequence_length=10):
    """Create sequence-based features for time series modeling"""
    df = data.copy()
    
    # Important columns for sequence features
    price_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
    indicator_cols = ['RSI', 'MACD', 'BB_Position', 'Stoch_K', 'ATR']
    
    # Create lagged features
    for col in price_cols + indicator_cols:
        if col in df.columns:
            for i in range(1, sequence_length + 1):
                df[f'{col}_lag_{i}'] = df[col].shift(i)
    
    # Create momentum and volatility features
    for window in [3, 5, 10]:
        df[f'{target_col}_momentum_{window}'] = df[target_col] / df[target_col].shift(window) - 1
        df[f'{target_col}_volatility_{window}'] = df[target_col].rolling(window).std() / df[target_col].rolling(window).mean()
    
    return df

def prepare_features_xgboost(data, look_back=20, forecast_horizon=1):
    """Prepare features for XGBoost model"""
    df = data.copy()
    df = create_sequence_features(df, sequence_length=look_back)
    
    # Create target variables
    for i in range(1, forecast_horizon + 1):
        df[f'Target_{i}'] = df['Close'].shift(-i)
    
    # Drop rows with NaN values
    df = df.dropna()
    return df

def prepare_lstm_data(data, look_back=60, forecast_horizon=1):
    """Prepare data for LSTM model"""
    # Select features for LSTM
    features = ['Close', 'Volume', 'High', 'Low', 'Open']
    if 'RSI' in data.columns:
        features.append('RSI')
    if 'MACD' in data.columns:
        features.append('MACD')
    if 'BB_Position' in data.columns:
        features.append('BB_Position')
    
    # Fill missing values
    feature_data = data[features].fillna(method='ffill').fillna(method='bfill')
    
    # Scale the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(feature_data)
    
    # Create sequences
    X, y = [], []
    for i in range(look_back, len(scaled_data) - forecast_horizon + 1):
        X.append(scaled_data[i-look_back:i])
        y.append(scaled_data[i:i+forecast_horizon, 0])  # Close price is at index 0
    
    X, y = np.array(X), np.array(y)
    if forecast_horizon == 1:
        y = y.reshape(-1)
    
    return X, y, scaler

## Data Preparation Testing

if sample_data is not None:
    data, company_name = sample_data
    data_with_indicators = add_technical_indicators(data)
    
    print("Testing XGBoost data preparation...")
    xgb_data = prepare_features_xgboost(data_with_indicators, look_back=20)
    print(f"XGBoost dataset shape: {xgb_data.shape}")
    print(f"Available features: {len([col for col in xgb_data.columns if not col.startswith('Target')])}")
    
    if LSTM_AVAILABLE:
        print("\nTesting LSTM data preparation...")
        X_lstm, y_lstm, lstm_scaler = prepare_lstm_data(data_with_indicators, look_back=60)
        print(f"LSTM X shape: {X_lstm.shape}")
        print(f"LSTM y shape: {y_lstm.shape}")
        print(f"LSTM features: {X_lstm.shape[2]}")
    
    # Display feature importance preview
    feature_cols = [col for col in xgb_data.columns if not col.startswith('Target') and 
                   col not in ['Open', 'Close', 'High', 'Low', 'Volume', 'Adj Close']]
    print(f"\nSample features for modeling:")
    print(f"Total features: {len(feature_cols)}")
    print(f"First 10 features: {feature_cols[:10]}")

## Data Quality Checks

def check_data_quality(data):
    """Check data quality and print summary"""
    print(f"Data Quality Report:")
    print(f"Total rows: {len(data)}")
    print(f"Total columns: {len(data.columns)}")
    print(f"Missing values per column:")
    
    missing_counts = data.isnull().sum()
    missing_pct = (missing_counts / len(data)) * 100
    
    for col in data.columns:
        if missing_counts[col] > 0:
            print(f"  {col}: {missing_counts[col]} ({missing_pct[col]:.1f}%)")
    
    print(f"Date range: {data.index[0]} to {data.index[-1]}")
    print(f"Business days: {len(data)}")
    
    return missing_counts, missing_pct

if sample_data is not None:
    print("\n" + "="*50)
    print("DATA QUALITY ANALYSIS")
    print("="*50)
    check_data_quality(data_with_indicators)

## Model Training Functions

In [20]:
# XGBoost ensemble model implementation

def train_xgboost_model(data, forecast_horizon=1):
    print("\nTraining Enhanced XGBoost model...")

    target_cols = [f'Target_{i}' for i in range(1, forecast_horizon + 1)]
    feature_cols = [col for col in data.columns if col not in target_cols + ['Open', 'Close', 'High', 'Low', 'Volume', 'Adj Close']]

    X = data[feature_cols]
    y = data[target_cols[0]] if len(target_cols) == 1 else data[target_cols]

    X = X.select_dtypes(include=[np.number])
    X = X.fillna(X.mean())

    split_ratio = 0.85
    split_idx = int(len(X) * split_ratio)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    scaler = RobustScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    models = []

    xgb_model = xgb.XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.03,
        subsample=0.9,
        colsample_bytree=0.9,
        reg_alpha=0.05,
        reg_lambda=0.05,
        random_state=42,
        n_jobs=-1
    )

    rf_model = RandomForestRegressor(
        n_estimators=200,
        max_depth=10,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )

    gb_model = GradientBoostingRegressor(
        n_estimators=200,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        random_state=42
    )

    models = [('XGB', xgb_model), ('RF', rf_model), ('GB', gb_model)]

    ensemble_train_pred = np.zeros(len(y_train))
    ensemble_test_pred = np.zeros(len(y_test))

    for name, model in models:
        model.fit(X_train_scaled, y_train)
        train_pred = model.predict(X_train_scaled)
        test_pred = model.predict(X_test_scaled)

        ensemble_train_pred += train_pred / len(models)
        ensemble_test_pred += test_pred / len(models)

    train_rmse = np.sqrt(mean_squared_error(y_train, ensemble_train_pred))
    test_rmse = np.sqrt(mean_squared_error(y_test, ensemble_test_pred))
    train_r2 = r2_score(y_train, ensemble_train_pred)
    test_r2 = r2_score(y_test, ensemble_test_pred)

    print(f"Ensemble Training RMSE: {train_rmse:.4f}")
    print(f"Ensemble Testing RMSE: {test_rmse:.4f}")
    print(f"Ensemble Training R²: {train_r2:.4f}")
    print(f"Ensemble Testing R²: {test_r2:.4f}")

    return models, X_test_scaled, y_test.values, ensemble_test_pred, split_idx, scaler

def prepare_features_xgboost(data, look_back=20, forecast_horizon=1):
    df = data.copy()
    df = create_sequence_features(df, sequence_length=look_back)

    for i in range(1, forecast_horizon + 1):
        df[f'Target_{i}'] = df['Close'].shift(-i)

    df = df.dropna()
    return df

def create_sequence_features(data, target_col='Close', sequence_length=10):
    df = data.copy()

    price_cols = ['Close', 'High', 'Low', 'Open', 'Volume']
    indicator_cols = ['RSI', 'MACD', 'BB_Position', 'Stoch_K', 'ATR']

    for col in price_cols + indicator_cols:
        if col in df.columns:
            for i in range(1, sequence_length + 1):
                df[f'{col}_lag_{i}'] = df[col].shift(i)

    for window in [3, 5, 10]:
        df[f'{target_col}_momentum_{window}'] = df[target_col] / df[target_col].shift(window) - 1
        df[f'{target_col}_volatility_{window}'] = df[target_col].rolling(window).std() / df[target_col].rolling(window).mean()

    return df

# Enhanced model selection and training logic
def get_model_choice():
    print("\nChoose prediction model:")
    print("1. XGBoost (Less time, High accuracy)")
    print("2. LSTM (More time, Most accuracy)")
    print("3. Both (Conclusion with both models)")

    while True:
        choice = input("Enter your choice (1/2/3): ")
        if choice in ['1', '2', '3']:
            return int(choice)
        print("Invalid choice. Please enter 1, 2, or 3.")

# Updated main function logic for XGBoost model
def main():
    print("=== Enhanced Stock Price Prediction System ===")

    stock_ticker = get_valid_ticker()
    model_choice = get_model_choice()

    result = fetch_stock_data(stock_ticker)
    if result is None:
        return

    data, company_name = result
    data_with_indicators = add_technical_indicators(data)

    xgb_results = None
    lstm_results = None

    if model_choice in [1, 3]:
        xgb_data = prepare_features_xgboost(data_with_indicators)
        xgb_results = train_xgboost_model(xgb_data)

    if model_choice in [2, 3] and LSTM_AVAILABLE:
        lstm_results = train_lstm_model(data_with_indicators)

    if model_choice == 1 and xgb_results[0] is not None:
        models, X_test, y_test, predictions, split_idx, scaler = xgb_results
        data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
        model_name = "Enhanced Ensemble"

In [21]:
# LSTM model implementation

def prepare_lstm_data(data, look_back=60, forecast_horizon=1):
    features = ['Close', 'Volume', 'High', 'Low', 'Open']
    if 'RSI' in data.columns:
        features.append('RSI')
    if 'MACD' in data.columns:
        features.append('MACD')
    if 'BB_Position' in data.columns:
        features.append('BB_Position')

    feature_data = data[features].fillna(method='ffill').fillna(method='bfill')

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(feature_data)

    X, y = [], []
    for i in range(look_back, len(scaled_data) - forecast_horizon + 1):
        X.append(scaled_data[i-look_back:i])
        y.append(scaled_data[i:i+forecast_horizon, 0])

    X, y = np.array(X), np.array(y)
    if forecast_horizon == 1:
        y = y.reshape(-1)

    return X, y, scaler

def train_lstm_model(data, forecast_horizon=1):
    if not LSTM_AVAILABLE:
        print("LSTM model not available. Skipping...")
        return None, None, None, None, None, None

    print("\nTraining Enhanced LSTM model...")

    X, y, scaler = prepare_lstm_data(data, look_back=60, forecast_horizon=forecast_horizon)

    split_ratio = 0.85
    split_idx = int(len(X) * split_ratio)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    model = Sequential([
        LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        BatchNormalization(),
        LSTM(64, return_sequences=True),
        Dropout(0.2),
        BatchNormalization(),
        GRU(32, return_sequences=False),
        Dropout(0.2),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.1),
        Dense(32, activation='relu'),
        Dense(forecast_horizon if forecast_horizon > 1 else 1)
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='huber',
        metrics=['mae']
    )

    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=10, min_lr=0.00001)

    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=150,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )

    train_pred = model.predict(X_train, verbose=0)
    test_pred = model.predict(X_test, verbose=0)

    if forecast_horizon == 1:
        train_pred = train_pred.flatten()
        test_pred = test_pred.flatten()

    train_pred_prices = scaler.inverse_transform(
        np.column_stack([train_pred, np.zeros((len(train_pred), scaler.n_features_in_ - 1))])
    )[:, 0]
    test_pred_prices = scaler.inverse_transform(
        np.column_stack([test_pred, np.zeros((len(test_pred), scaler.n_features_in_ - 1))])
    )[:, 0]
    y_train_prices = scaler.inverse_transform(
        np.column_stack([y_train, np.zeros((len(y_train), scaler.n_features_in_ - 1))])
    )[:, 0]
    y_test_prices = scaler.inverse_transform(
        np.column_stack([y_test, np.zeros((len(y_test), scaler.n_features_in_ - 1))])
    )[:, 0]

    train_rmse = np.sqrt(mean_squared_error(y_train_prices, train_pred_prices))
    test_rmse = np.sqrt(mean_squared_error(y_test_prices, test_pred_prices))
    train_r2 = r2_score(y_train_prices, train_pred_prices)
    test_r2 = r2_score(y_test_prices, test_pred_prices)

    print(f"LSTM Training RMSE: {train_rmse:.4f}")
    print(f"LSTM Testing RMSE: {test_rmse:.4f}")
    print(f"LSTM Training R²: {train_r2:.4f}")
    print(f"LSTM Testing R²: {test_r2:.4f}")

    return model, X_test, y_test_prices, test_pred_prices, split_idx + 60, scaler

try:
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, GRU
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
    LSTM_AVAILABLE = True
except ImportError:
    print("TensorFlow not available. LSTM model will be skipped.")
    LSTM_AVAILABLE = False

# Updated main function for LSTM handling
def main():
    print("=== Enhanced Stock Price Prediction System ===")

    stock_ticker = get_valid_ticker()
    model_choice = get_model_choice()

    result = fetch_stock_data(stock_ticker)
    if result is None:
        return

    data, company_name = result
    data_with_indicators = add_technical_indicators(data)

    xgb_results = None
    lstm_results = None

    if model_choice in [1, 3]:
        xgb_data = prepare_features_xgboost(data_with_indicators)
        xgb_results = train_xgboost_model(xgb_data)

    if model_choice in [2, 3] and LSTM_AVAILABLE:
        lstm_results = train_lstm_model(data_with_indicators)

    if model_choice == 1 and xgb_results[0] is not None:
        models, X_test, y_test, predictions, split_idx, scaler = xgb_results
        data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
        model_name = "Enhanced Ensemble"

    elif model_choice == 2 and lstm_results[0] is not None:
        model, X_test, y_test, predictions, split_idx, scaler = lstm_results
        data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
        model_name = "Enhanced LSTM"

# Trading signals generation and recommendation system

In [ ]:
def generate_trading_signals(original_data, predictions, start_idx):
    df = original_data.copy()
    df['Predicted'] = np.nan
    df['Buy_Signal'] = False
    df['Sell_Signal'] = False

    if len(predictions) > 0:
        end_idx = start_idx + len(predictions)
        if end_idx <= len(df):
            df.iloc[start_idx:end_idx, df.columns.get_loc('Predicted')] = predictions

    if 'BB_Upper' not in df.columns:
        df['BB_Upper'] = df['Close'].rolling(window=20).mean() + (df['Close'].rolling(window=20).std() * 2)
        df['BB_Lower'] = df['Close'].rolling(window=20).mean() - (df['Close'].rolling(window=20).std() * 2)
        df['BB_Middle'] = df['Close'].rolling(window=20).mean()

    df['Signal'] = 0

    for i in range(max(1, start_idx), len(df)):
        if pd.notna(df.iloc[i]['Predicted']):
            current_price = df.iloc[i]['Close']
            predicted_price = df.iloc[i]['Predicted']
            bb_lower = df.iloc[i]['BB_Lower']
            bb_upper = df.iloc[i]['BB_Upper']
            rsi = df.iloc[i].get('RSI', 50)

            price_change = (predicted_price - current_price) / current_price

            if (current_price <= bb_lower * 1.01 and
                predicted_price > current_price * 1.002 and
                rsi < 45 and price_change > 0.005):
                df.iloc[i, df.columns.get_loc('Buy_Signal')] = True
                df.iloc[i, df.columns.get_loc('Signal')] = 1
            elif (current_price >= bb_upper * 0.99 and
                  predicted_price < current_price * 0.998 and
                  rsi > 55 and price_change < -0.005):
                df.iloc[i, df.columns.get_loc('Sell_Signal')] = True
                df.iloc[i, df.columns.get_loc('Signal')] = -1

    return df

def calculate_recommendation(data, current_price, predicted_price, rsi, bb_position):
    signals = []

    price_change = (predicted_price - current_price) / current_price
    if price_change > 0.02:
        signals.append(('buy', 0.4))
    elif price_change < -0.02:
        signals.append(('sell', 0.4))
    else:
        signals.append(('hold', 0.3))

    if rsi < 30:
        signals.append(('buy', 0.3))
    elif rsi > 70:
        signals.append(('sell', 0.3))
    else:
        signals.append(('hold', 0.2))

    if bb_position < 0.2:
        signals.append(('buy', 0.25))
    elif bb_position > 0.8:
        signals.append(('sell', 0.25))
    else:
        signals.append(('hold', 0.2))

    buy_prob = sum([weight for action, weight in signals if action == 'buy'])
    sell_prob = sum([weight for action, weight in signals if action == 'sell'])
    hold_prob = sum([weight for action, weight in signals if action == 'hold'])

    total = buy_prob + sell_prob + hold_prob
    buy_prob = (buy_prob / total) * 100
    sell_prob = (sell_prob / total) * 100
    hold_prob = (hold_prob / total) * 100

    return buy_prob, hold_prob, sell_prob

# Updated main function for combined model handling
def main():
    print("=== Enhanced Stock Price Prediction System ===")

    stock_ticker = get_valid_ticker()
    model_choice = get_model_choice()

    result = fetch_stock_data(stock_ticker)
    if result is None:
        return

    data, company_name = result
    data_with_indicators = add_technical_indicators(data)

    xgb_results = None
    lstm_results = None

    if model_choice in [1, 3]:
        xgb_data = prepare_features_xgboost(data_with_indicators)
        xgb_results = train_xgboost_model(xgb_data)

    if model_choice in [2, 3] and LSTM_AVAILABLE:
        lstm_results = train_lstm_model(data_with_indicators)

    if model_choice == 1 and xgb_results[0] is not None:
        models, X_test, y_test, predictions, split_idx, scaler = xgb_results
        data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
        model_name = "Enhanced Ensemble"

    elif model_choice == 2 and lstm_results[0] is not None:
        model, X_test, y_test, predictions, split_idx, scaler = lstm_results
        data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
        model_name = "Enhanced LSTM"

    elif model_choice == 3:
        if xgb_results[0] is not None and lstm_results[0] is not None:
            xgb_pred = xgb_results[3]
            lstm_pred = lstm_results[3]

            min_len = min(len(xgb_pred), len(lstm_pred))
            xgb_weight = 0.6
            lstm_weight = 0.4
            avg_predictions = (xgb_pred[:min_len] * xgb_weight + lstm_pred[:min_len] * lstm_weight)

            split_idx = max(xgb_results[4], lstm_results[4])
            data_with_signals = generate_trading_signals(data_with_indicators, avg_predictions, split_idx)
            model_name = "Combined Enhanced Models"
        else:
            print("Both models not available. Using available model.")
            if xgb_results[0] is not None:
                models, X_test, y_test, predictions, split_idx, scaler = xgb_results
                data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
                model_name = "Enhanced Ensemble"
            else:
                return

    current_price = data_with_signals['Close'].iloc[-1]
    predicted_price = data_with_signals['Predicted'].iloc[-1] if pd.notna(data_with_signals['Predicted'].iloc[-1]) else current_price
    current_rsi = data_with_indicators['RSI'].iloc[-1] if 'RSI' in data_with_indicators.columns else 50

    if 'BB_Upper' in data_with_signals.columns and 'BB_Lower' in data_with_signals.columns:
        bb_upper = data_with_signals['BB_Upper'].iloc[-1]
        bb_lower = data_with_signals['BB_Lower'].iloc[-1]
        bb_position = (current_price - bb_lower) / (bb_upper - bb_lower) if bb_upper != bb_lower else 0.5
    else:
        bb_position = 0.5

    buy_prob, hold_prob, sell_prob = calculate_recommendation(
        data_with_signals, current_price, predicted_price, current_rsi, bb_position
    )

    print(f"\n=== {model_name} Model Results ===")
    print(f"Current Price: ${current_price:.2f}")
    print(f"Predicted Next Price: ${predicted_price:.2f}")
    print(f"Price Change: {((predicted_price - current_price) / current_price) * 100:.2f}%")
    print(f"Current RSI: {current_rsi:.2f}")

    print(f"\n=== Trading Recommendation ===")
    print(f"BUY Probability: {buy_prob:.1f}%")
    print(f"HOLD Probability: {hold_prob:.1f}%")
    print(f"SELL Probability: {sell_prob:.1f}%")

    if buy_prob > max(hold_prob, sell_prob):
        main_rec = "BUY"
    elif sell_prob > max(buy_prob, hold_prob):
        main_rec = "SELL"
    else:
        main_rec = "HOLD"

    print(f"\nMain Recommendation: {main_rec}")

## Visualization Functions

In [ ]:
def create_interactive_plot(data, company_name, ticker_symbol):
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=[f'{company_name} ({ticker_symbol}) - Stock Analysis', 'Volume'],
        vertical_spacing=0.1,
        row_heights=[0.7, 0.3]
    )
    
    # Add price trace
    fig.add_trace(
        go.Scatter(x=data.index, y=data['Close'],
                  name='Stock Price', line=dict(color='blue', width=2)),
        row=1, col=1
    )
    
    # Add volume trace  
    fig.add_trace(
        go.Bar(x=data.index, y=data['Volume'],
               name='Volume', marker_color='rgba(0,100,80,0.6)'),
        row=2, col=1
    )
    
    fig.update_layout(
        title=f'{company_name} ({ticker_symbol}) - Stock Analysis Dashboard',
        height=800,
        showlegend=True
    )
    
    fig.show()

## Main Execution

In [ ]:
def main():
    print("=== Enhanced Stock Price Prediction System ===")

    stock_ticker = get_valid_ticker()
    model_choice = get_model_choice()

    result = fetch_stock_data(stock_ticker)
    if result is None:
        return

    data, company_name = result
    data_with_indicators = add_technical_indicators(data)

    xgb_results = None
    lstm_results = None

    if model_choice in [1, 3]:
        xgb_data = prepare_features_xgboost(data_with_indicators)
        xgb_results = train_xgboost_model(xgb_data)

    if model_choice in [2, 3] and LSTM_AVAILABLE:
        lstm_results = train_lstm_model(data_with_indicators)

    if model_choice == 1 and xgb_results[0] is not None:
        models, X_test, y_test, predictions, split_idx, scaler = xgb_results
        data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
        model_name = "Enhanced Ensemble"

    elif model_choice == 2 and lstm_results[0] is not None:
        model, X_test, y_test, predictions, split_idx, scaler = lstm_results
        data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
        model_name = "Enhanced LSTM"

    elif model_choice == 3:
        if xgb_results[0] is not None and lstm_results[0] is not None:
            xgb_pred = xgb_results[3]
            lstm_pred = lstm_results[3]

            min_len = min(len(xgb_pred), len(lstm_pred))
            xgb_weight = 0.6
            lstm_weight = 0.4
            avg_predictions = (xgb_pred[:min_len] * xgb_weight + lstm_pred[:min_len] * lstm_weight)

            split_idx = max(xgb_results[4], lstm_results[4])
            data_with_signals = generate_trading_signals(data_with_indicators, avg_predictions, split_idx)
            model_name = "Combined Enhanced Models"
        else:
            print("Both models not available. Using available model.")
            if xgb_results[0] is not None:
                models, X_test, y_test, predictions, split_idx, scaler = xgb_results
                data_with_signals = generate_trading_signals(data_with_indicators, predictions, split_idx)
                model_name = "Enhanced Ensemble"
            else:
                return

    current_price = data_with_signals['Close'].iloc[-1]
    predicted_price = data_with_signals['Predicted'].iloc[-1] if pd.notna(data_with_signals['Predicted'].iloc[-1]) else current_price
    current_rsi = data_with_indicators['RSI'].iloc[-1] if 'RSI' in data_with_indicators.columns else 50

    if 'BB_Upper' in data_with_signals.columns and 'BB_Lower' in data_with_signals.columns:
        bb_upper = data_with_signals['BB_Upper'].iloc[-1]
        bb_lower = data_with_signals['BB_Lower'].iloc[-1]
        bb_position = (current_price - bb_lower) / (bb_upper - bb_lower) if bb_upper != bb_lower else 0.5
    else:
        bb_position = 0.5

    buy_prob, hold_prob, sell_prob = calculate_recommendation(
        data_with_signals, current_price, predicted_price, current_rsi, bb_position
    )

    print(f"\n=== {model_name} Model Results ===")
    print(f"Current Price: ${current_price:.2f}")
    print(f"Predicted Next Price: ${predicted_price:.2f}")
    print(f"Price Change: {((predicted_price - current_price) / current_price) * 100:.2f}%")
    print(f"Current RSI: {current_rsi:.2f}")

    print(f"\n=== Trading Recommendation ===")
    print(f"BUY Probability: {buy_prob:.1f}%")
    print(f"HOLD Probability: {hold_prob:.1f}%")
    print(f"SELL Probability: {sell_prob:.1f}%")

    if buy_prob > max(hold_prob, sell_prob):
        main_rec = "BUY"
    elif sell_prob > max(buy_prob, hold_prob):
        main_rec = "SELL"
    else:
        main_rec = "HOLD"

    print(f"\nMain Recommendation: {main_rec}")

    create_interactive_plot(data_with_signals, company_name, stock_ticker)

    print(f"\nEnhanced Analysis complete! Interactive chart displayed above.")

if __name__ == "__main__":
    main()

Stock Prediction System initialized
